<h1>Importing Necessary Packages</h1>

In [ ]:
import sys
sys.path.append("../")
import os
import ast
from app.configs import *
from scripts.fmd import FMDCalculator
import numpy as np
import pandas as pd
import time

<h1>Load $\Delta_\text{FMD}$ cache</h1>

If you don't have a cache, create it using create_fmd_map.ipynb

In [ ]:
try:
    files = os.listdir(cfg_mesh_directory)
except FileNotFoundError:
    raise Exception('Directory not found! Check configs.py and put proper cfg_mesh_directory')
xml_file = None
for f in files:
    if f.endswith('.xml'):
        xml_file = f
        print(f"XML file found: {cfg_mesh_directory}/{f}")
if xml_file is None:
    raise Exception('No xml file found! Check configs.py and put proper cfg_mesh_directory')
    
fmd = FMDCalculator(
    arena_size=(cfg_scene_length, cfg_scene_breadth), spacing=0.5,
    real_csv_path=cfg_cir_path,
    scene_path=f"{cfg_mesh_directory}/{xml_file}"
)
fmd.import_cache_from_json(cfg_fmd_cache_name)
tx_list = [ast.literal_eval(k) for k in fmd._cache.keys()]
pseudo_transmitter = tx_list[0]

<h1>Run Benchmarking on your machine</h1>

You can find the benchmark data in app/benchmark.csv. Executing below cell overwrites existing benchmark if any

In [ ]:
bitmask = np.zeros((cfg_scene_breadth*int(1/0.5),cfg_scene_length*int(1/0.5)))
N = cfg_scene_breadth*int(1/0.5)*cfg_scene_length*int(1/0.5)

rows = []

for p in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5]:
    bm = np.zeros_like(bitmask)
    bm.flat[np.random.choice(N, int(p * N), replace=False)] = 1
    fmdcalc, timers = fmd.update_fmd(pseudo_transmitter)
    row = {"percentage": p}
    print("Benchmarking: ",(p*100),"%")
    row.update(timers)
    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv("benchmark.csv", index=False)
